In [1]:
import tensorflow as tf
import numpy as np
import tensorlayer as tl
from tensorlayer.layers import *

In [2]:
import matplotlib.pyplot as plt  # dealing with plots 
import os
from scipy.misc import imread

In [3]:
def load_image_path(data_path, valid_proportion, test_proportion, pos_path="1/", neg_path="0/"):
    """
    Args:
        data_path: list,数据所在文件夹，最后有一杠
        valid_proportion: float，验证集所占百分比，小数，如0.1
        test_proportion: float，测试集所占百分比
        pos_path: str,正样本所在文件夹
        neg_path: str,负样本所在文件夹
    Returns:
        x_train: list，dtype=str，图片路径
        y_train: list, dtype=int
        x_valid: list，dtype=str，图片路径
        y_valid: list, dtype=int
        x_test: list，dtype=str，图片路径
        y_test: list, dtype=int
    """

    pos_image_path = []
    pos_labels = []

    neg_image_path = []
    neg_labels = []

    ful_image_path = []
    ful_labels = []

    np.random.seed(0)

    pos_path = data_path + pos_path
    for img in tf.gfile.ListDirectory(pos_path):
        label = 1

        path = os.path.join(pos_path, img)
        pos_image_path.append(path)
        pos_labels.append(label)

    neg_path = data_path + neg_path
    for img in tf.gfile.ListDirectory(neg_path):
        label = 0

        path = os.path.join(neg_path, img)
        neg_image_path.append(path)
        neg_labels.append(label)

    ful_image_path = pos_image_path + neg_image_path
    ful_labels = pos_labels + neg_labels

    temp = np.array([ful_image_path, ful_labels])
    temp = temp.transpose()
    np.random.shuffle(temp)
    ful_image_path = list(temp[:, 0])
    ful_labels = list(temp[:, 1])
    ful_labels = [int(i) for i in ful_labels]

    x_valid = []
    y_valid = []
    x_test = []
    y_test = []
    from sklearn.model_selection import train_test_split
    if not valid_proportion == 0:
        x_train, x_valid, y_train, y_valid = train_test_split(ful_image_path, ful_labels,
                                                              test_size=(valid_proportion + test_proportion),
                                                              stratify=ful_labels, random_state=1)
        if not test_proportion == 0:
            x_valid, x_test, y_valid, y_test = train_test_split(x_valid, y_valid, test_size=test_proportion / (
                    valid_proportion + test_proportion), stratify=y_valid, random_state=1)
    else:
        x_train = ful_image_path
        y_train = ful_labels

    print("train_num: %d ,pos_num: %d , neg_num: %d" % (
        len(y_train), y_train.count(1), len(y_train) - y_train.count(1)))
    print("valid_num: %d ,pos_num: %d , neg_num: %d" % (
        len(y_valid), y_valid.count(1), len(y_valid) - y_valid.count(1)))
    print("test_num : %d ,pos_num: %d , neg_num: %d" % (
        len(y_test), y_test.count(1), len(y_test) - y_test.count(1)))

    return x_train, y_train, x_valid, y_valid, x_test, y_test


def get_batch(x, y, image_W, image_H, batch_size, capacity=500, epochs=None):
    '''
    Args:
        x: list type
        y: list type
        image_W: image width
        image_H: image height
        batch_size: batch size
        capacity: the maximum elements in queue
        epochs: 如果要指定epoch数
    Returns:
        image_batch: 4D tensor [batch_size, width, height, 3], dtype=tf.float32
        label_batch: 1D tensor [batch_size], dtype=tf.int32
    '''

    x = tf.cast(x, tf.string)
    y = tf.cast(y, tf.int32)

    # make an input queue
    input_queue = tf.train.slice_input_producer([x, y], num_epochs=epochs)

    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0])
    image = tf.image.decode_jpeg(image_contents, channels=3)

    ######################################
    # data argumentation should go to here
    ######################################

    image = tf.image.resize_images(image, [image_W, image_H], method=0)
    # image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)
    # if you want to test the generated batches of images, you might want to comment the following line.
    # 如果想看到正常的图片，请注释掉111行（标准化）和 126行（image_batch = tf.cast(image_batch, tf.float32)）
    # 训练时不要注释掉！
    image = tf.image.per_image_standardization(image)

    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,
                                              capacity=capacity)

    image_batch = tf.cast(image_batch, tf.float32)
    label_batch = tf.reshape(label_batch, [batch_size])

    # label_batch = to_categorical(y=label_batch, nb_classes=2)
    return image_batch, label_batch

In [4]:
Train_Proportion = 0.9  # Proportion of the data to be used for training
Valid_Proportion = 0.1
Test_Proportion = 0  #
data_dir = "F:/Python/data/food_101_dataset/"
x_train, y_train, x_valid, y_valid, x_test, y_test = load_image_path(data_dir, Valid_Proportion,
                                                                                Test_Proportion)

train_num: 4042 ,pos_num: 852 , neg_num: 3190
valid_num: 450 ,pos_num: 95 , neg_num: 355
test_num : 0 ,pos_num: 0 , neg_num: 0


In [5]:
#写了一个函数，用来自动获取batchsize的合适值。使得total_num % batchsize = 0
def get_batch_size(x):
    #首先求出因数
    div = []
    i = 1
    while i <= x :
        if x % i == 0:
            div.append(i)
        i += 1    
    #把因数list反转，然后找一个最好是100到300之间的最大值，没有那就0到100之间的最大值。都没有就raise
    div.reverse()
    for b in div:
        if b - 100 < 200 or 100 - b >0 :
            return b

    raise NameError("No such Number") 

In [9]:
#注意，这里要把epoch设置成1
batch_size = get_batch_size(len(y_train))
print("batchsize : " + str(batch_size))
train_batch_run, train_label_batch_run = get_batch(x_train, y_train, 224, 224, batch_size, capacity=500, epochs=1)

batchsize : 94


In [19]:
pre_trained_dir = "F:/Python/pre-trained/"

In [14]:
#查看vgg的网络结构

#创建一个读写器
reader = tf.train.NewCheckpointReader(checkpoint_dir + "vgg_16.ckpt")
#读变量和shape的映射字典
all_variables_shape = reader.get_variable_to_shape_map()
#读变量和dtyte的映射字典
all_variables_dtyte = reader.get_variable_to_dtype_map()

for variable in all_variables_shape:
    #如果要看全部的变量就把下面这个if注释掉
    if "fc" in variable:
        print(variable," : ",all_variables_shape[variable])
# print(all_variables_shape)


vgg_16/fc8/weights  :  [1, 1, 4096, 1000]
vgg_16/fc7/biases  :  [4096]
vgg_16/fc7/weights  :  [1, 1, 4096, 4096]
vgg_16/fc6/weights  :  [7, 7, 512, 4096]
vgg_16/fc8/biases  :  [1000]
vgg_16/fc6/biases  :  [4096]


In [16]:
#定义一个VGG16的网络
def model(input):
    network = tl.layers.InputLayer(input, name='input')
    # conv1
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 3, 64],  # 64 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1_1')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 64, 64],  # 64 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv1_2')
    network = PoolLayer(network, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', pool=tf.nn.max_pool, name='pool1')

    # conv2
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 64, 128],  # 128 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv2_1')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 128, 128],  # 128 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv2_2')
    network = PoolLayer(network, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', pool=tf.nn.max_pool, name='pool2')

    # conv3
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 128, 256],  # 256 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv3_1')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 256, 256],  # 256 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv3_2')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 256, 256],  # 256 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv3_3')
    network = PoolLayer(network, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', pool=tf.nn.max_pool, name='pool3')

    # conv4
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 256, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv4_1')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 512, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv4_2')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 512, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv4_3')
    network = PoolLayer(network, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', pool=tf.nn.max_pool, name='pool4')

    # conv5
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 512, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv5_1')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 512, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv5_2')
    network = Conv2dLayer(
        network,
        act=tf.nn.relu,
        shape=[3, 3, 512, 512],  # 512 features for each 3x3 patch
        strides=[1, 1, 1, 1],
        padding='SAME',
        name='conv5_3')
    network = PoolLayer(network, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', pool=tf.nn.max_pool, name='pool5')
    network = FlattenLayer(network, name='flatten')
    network = DenseLayer(network, n_units=4096, act=tf.nn.relu, name='fc6_relu')
    network = DenseLayer(network, n_units=4096, act=tf.nn.relu, name='fc7_relu')
    y = network.outputs
    return  y, network
    

In [17]:
#进行inference
#要注意一点就是，当从tf.train.batch获得batch和batch_label后，我可以对其中一个进行计算，比如下面的
#model，将batch变成y_。不管我怎么叠加运算，只要我不对其中任何一个进行run，最后再同时run，就都是对应的两个变量
y_, net = model(train_batch_run)

  [TL] InputLayer  input: (94, 224, 224, 3)
  [TL] Conv2dLayer conv1_1: shape:[3, 3, 3, 64] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] Conv2dLayer conv1_2: shape:[3, 3, 64, 64] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] PoolLayer   pool1: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
  [TL] Conv2dLayer conv2_1: shape:[3, 3, 64, 128] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] Conv2dLayer conv2_2: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] PoolLayer   pool2: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
  [TL] Conv2dLayer conv3_1: shape:[3, 3, 128, 256] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] Conv2dLayer conv3_2: shape:[3, 3, 256, 256] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] Conv2dLayer conv3_3: shape:[3, 3, 256, 256] strides:[1, 1, 1, 1] pad:SAME act:relu
  [TL] PoolLayer   pool3: ksize:[1, 2, 2, 1] strides:[1, 2, 2, 1] padding:SAME pool:max_pool
  [TL] Conv2dLayer conv4_1: shape:[3, 3, 256, 512] st

In [18]:
#因为第一种迁移学习就是不改变前面的变量，只去掉最后一层全连接层。所以要载入模型的时候要有选择的载入。需要准备
#一个list或者dict，指定需要载入哪些变量
def prepare():
    conv1_1_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv1_1/W_conv2d:0")
    conv1_1_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv1_1/b_conv2d:0")
    conv1_2_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv1_2/W_conv2d:0")
    conv1_2_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv1_2/b_conv2d:0")

    conv2_1_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv2_1/W_conv2d:0")
    conv2_1_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv2_1/b_conv2d:0")
    conv2_2_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv2_2/W_conv2d:0")
    conv2_2_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv2_2/b_conv2d:0")

    conv3_1_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_1/W_conv2d:0")
    conv3_1_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_1/b_conv2d:0")
    conv3_2_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_2/W_conv2d:0")
    conv3_2_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_2/b_conv2d:0")
    conv3_3_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_3/W_conv2d:0")
    conv3_3_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv3_3/b_conv2d:0")

    conv4_1_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_1/W_conv2d:0")
    conv4_1_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_1/b_conv2d:0")
    conv4_2_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_2/W_conv2d:0")
    conv4_2_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_2/b_conv2d:0")
    conv4_3_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_3/W_conv2d:0")
    conv4_3_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv4_3/b_conv2d:0")

    conv5_1_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_1/W_conv2d:0")
    conv5_1_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_1/b_conv2d:0")
    conv5_2_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_2/W_conv2d:0")
    conv5_2_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_2/b_conv2d:0")
    conv5_3_W_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_3/W_conv2d:0")
    conv5_3_b_conv2d = tf.get_default_graph().get_tensor_by_name("conv5_3/b_conv2d:0")

    fc6_relu_W = tf.get_default_graph().get_tensor_by_name("fc6_relu/W:0")
    fc6_relu_b = tf.get_default_graph().get_tensor_by_name("fc6_relu/b:0")
    fc7_relu_W = tf.get_default_graph().get_tensor_by_name("fc7_relu/W:0")
    fc7_relu_b = tf.get_default_graph().get_tensor_by_name("fc7_relu/b:0")
    #fc8不要
    
    dict = {
        #左边是原模型的名字，右边是新模型的引用
        "vgg_16/conv1/conv1_1/weights": conv1_1_W_conv2d,
        "vgg_16/conv1/conv1_1/biases": conv1_1_b_conv2d,
        "vgg_16/conv1/conv1_2/weights": conv1_2_W_conv2d,
        "vgg_16/conv1/conv1_2/biases": conv1_2_b_conv2d,

        "vgg_16/conv2/conv2_1/weights": conv2_1_W_conv2d,
        "vgg_16/conv2/conv2_1/biases": conv2_1_b_conv2d,
        "vgg_16/conv2/conv2_2/weights": conv2_2_W_conv2d,
        "vgg_16/conv2/conv2_2/biases": conv2_2_b_conv2d,

        "vgg_16/conv3/conv3_1/weights": conv3_1_W_conv2d,
        "vgg_16/conv3/conv3_1/biases": conv3_1_b_conv2d,
        "vgg_16/conv3/conv3_2/weights": conv3_2_W_conv2d,
        "vgg_16/conv3/conv3_2/biases": conv3_2_b_conv2d,
        "vgg_16/conv3/conv3_3/weights": conv3_3_W_conv2d,
        "vgg_16/conv3/conv3_3/biases": conv3_3_b_conv2d,

        "vgg_16/conv4/conv4_1/weights": conv4_1_W_conv2d,
        "vgg_16/conv4/conv4_1/biases": conv4_1_b_conv2d,
        "vgg_16/conv4/conv4_2/weights": conv4_2_W_conv2d,
        "vgg_16/conv4/conv4_2/biases": conv4_2_b_conv2d,
        "vgg_16/conv4/conv4_3/weights": conv4_3_W_conv2d,
        "vgg_16/conv4/conv4_3/biases": conv4_3_b_conv2d,

        "vgg_16/conv5/conv5_1/weights": conv5_1_W_conv2d,
        "vgg_16/conv5/conv5_1/biases": conv5_1_b_conv2d,
        "vgg_16/conv5/conv5_2/weights": conv5_2_W_conv2d,
        "vgg_16/conv5/conv5_2/biases": conv5_2_b_conv2d,
        "vgg_16/conv5/conv5_3/weights": conv5_3_W_conv2d,
        "vgg_16/conv5/conv5_3/biases": conv5_3_b_conv2d,

        "vgg_16/fc6/weights": fc6_relu_W,
        "vgg_16/fc6/biases": fc6_relu_b,
        "vgg_16/fc7/weights": fc7_relu_W,
        "vgg_16/fc7/biases": fc7_relu_b
        }
    return dict
        

In [20]:
d = prepare()
# 读取ckpt里保存的参数
sess = tf.InteractiveSession()
#注意，你回头去看原模型和新模型的全连接层的shape不一样，所以下面可以开启reshape = True
saver = tf.train.Saver(max_to_keep=2, var_list=d, reshape=True)

saver.restore(sess, pre_trained_dir + "vgg_16.ckpt")
print("load ok!")

INFO:tensorflow:Restoring parameters from F:/Python/pre-trained/vgg_16.ckpt
load ok!


In [ ]:
final_dir = "C:/Users/hsqyc/Desktop/"
filename = final_dir + "dogVScat_train_vgg_tfrecord"

print("Converting data into %s ..." % filename)
#     with tf.Session() as sess:

writer = tf.python_io.TFRecordWriter(filename)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)
i = 0
try:
    while not coord.should_stop():
        A, B = sess.run([y_ ,train_label_batch_run])
        for index in range(len(B)):
            image = A[index]
            image = image.reshape(4096, )
            img_raw = image.tostring()  # tobytes()

            label = int(B[index])
            #             img_raw = image.tostring()#tobytes()

            example = tf.train.Example(features=tf.train.Features(feature={
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                #                 "img_w": tf.train.Feature(int64_list=tf.train.Int64List(value=[img_w])),
                #                 "img_h": tf.train.Feature(int64_list=tf.train.Int64List(value=[img_h])),
                "img_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            }))
            writer.write(example.SerializeToString())  # Serialize To String
            print(i)
            i += 1
except tf.errors.OutOfRangeError:
    print('done!')
finally:
    coord.request_stop()
writer.close()
print("done")

In [ ]:
tf.assign("conv1_1/b_conv2d",reader.get_tensor("vgg_16/conv1/conv1_2/biases"))